In [2]:
import tensorflow as tf
import cv2
import sys
import numpy as np
from tqdm import tqdm

from efficientnet.tfkeras import EfficientNetB4, preprocess_input
import pandas as pd
import os
from PIL import Image
import glob
import xml.etree.ElementTree as ET
from xml.dom import minidom
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model

In [255]:
import tensorflow as tf
import cv2
import sys
import numpy as np
from tqdm import tqdm
from tensorflow.keras.models import load_model
#from efficientnet.tfkeras import EfficientNetB4, preprocess_input
from efficientnet.tfkeras import EfficientNetB4, preprocess_input
import pandas as pd
import os
import glob
from keras_adabound import AdaBound  # optimizer


IMG_RESIZE = 380
#IMG_RESIZE = 456

model_dir = './model'

data_dir = './landmark_dataset'
model_name = 'landmark_efnb4_380_cv4_adabound_v1'


model_path = os.path.join(model_dir, model_name, model_name + '.h5')

# submission
df_sub = pd.read_csv('./landmark_dataset/public/sample_submisstion.csv', sep = ',', index_col=False)

test_image_dir = './landmark_dataset/test/'
image_name = df_sub['id'].tolist()
df_cate = pd.read_csv('./landmark_dataset/public/category.csv')

# Label Load
df = pd.read_csv('./Label.txt', sep = '\n', index_col=False, header=None)
CLASS_NAMES = df[0].tolist()
CLASS_NAMES = sorted(CLASS_NAMES)
print(len(CLASS_NAMES))

# Model Load
#model = tf.keras.models.load_model(model_path)
model = tf.keras.models.load_model(model_path, custom_objects={'AdaBound' : AdaBound})  # adabound Ver
print(model.summary())


def infer(image_path, model_name, class_file):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_RESIZE, IMG_RESIZE))

    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    predictions = model_name.predict(image, steps=1)
    score = np.argmax(predictions[0])

    result = class_file[score]
    prob = predictions[0].max()
        
    return result, prob


final_result = []
for image in tqdm(image_name):
    num = image[0]
    image_path = test_image_dir + num + '/' + image + '.JPG'
    
    result = infer(image_path = image_path, model_name = model, class_file=CLASS_NAMES)
    
    landmark_id = df_cate[df_cate['landmark_name'] == result[0]]['landmark_id'].values[0]
    
    final_result.append({'id' : image, 'landmark_id' : landmark_id, 'conf' : result[1]})


df = pd.DataFrame(final_result)
df.to_csv(f'./sub/{model_name}.csv', index=False)

1049


  0%|          | 0/37964 [00:00<?, ?it/s]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b4 (Model)      (None, 12, 12, 1792)      17673816  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               459008    
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1049)              269593    
Total params: 18,403,441
Trainable params: 18,277,729
Non-trainable params: 125,712
______________________________________

100%|██████████| 37964/37964 [55:08<00:00, 11.48it/s]  


In [253]:
df[df['conf'] <= 0.8]

KeyError: 'conf'

In [121]:
result = []
idx = 0
for label in label_list:
    if label[-1] == ']':
        label = label.replace('[', '[[]')
        find_index = label.rfind(']')
        label = label[:find_index] + '[]]' + label[find_index+1:]
    print(label)    
    file_list = glob(os.path.join('./landmark_dataset/train',label,'*'))
    for file in file_list:
        result.append([idx, label, file])
        idx += 1
        
img_df = pd.DataFrame(result, columns=['idx','label','image_path'])


서울대학교병원
종로신진시장
궐리사
동구릉혜릉
해운대교육지원청
인천광역시교육청북구도서관
지장암
평택시립 배다리 도서관
킨텍스
태조 이성계상
전차381호
시흥대교
시흥시청
양씨삼강문
숭실대학교
부산지방우정청
온천엘리바덴
절두산 순교 성지
인천학생교육문화회관
부산금사요양병원
역대현감비석군
오크우드 프리미어 인천 호텔
반여도서관
석촌동 고분군
수영구 생활문화센터
혜명단청박물관
모아뮤지엄
일산시장
화석정
한국이민사박물관
부천시청
인천성모병원
배다골 테마파크
신촌세브란스병원
천주교이기대성당
현대시장
덕소 도서관
오이도 빨강등대
수유전통시장
수성구립 고산도서관
후암시장
올림픽공원 세계평화의문
롯데백화점[[]건대스타시티[]]
밀레니엄 힐튼 서울
안양 중초사지 삼층석탑
전태일 기념관
동국대학교 (바이오메디 캠퍼스)
중남미 문화원
이천 세라피아
메이필드 호텔 앤 리조트
구로시장
부천 옹기 박물관
모래내시장
현대백화점[[]본점[]]
환구단
광주 남구 청소년 수련관(문화정보 도서관, 문화원, 남구 생활문화센터 포함)
부평향교
청운문학도서관
파크하얏트 서울
부산환경공단남부사업소
더 클래식 500
진흥종합시장
강남대학교
송정동 행정복지센터
인천광역시 여성문화회관
시흥 오이도 선사 유적 공원
대화 도서관
용비교
인천 구 대화조 사무소
강남스타일 말춤 손목 조형물
롯데백화점[[]캐슬플라자[]]
망원정지
유니온파크 물놀이터
동백해녀복지회관
BIFC
홍예문
의정부 정보도서관
NC백화점[[]송파[]]
포시즌스 호텔 서울
한국외국어대학교 서울캠퍼스
서울풍물시장
송도해돋이도서관
인천 해돋이공원
인천시역사자료관
동작도서관
양재도서관
경운궁중명전
가락시장
영종도서관
백범김구선생상
뚝섬한강공원
천호대교
윤동주 문학관
가회민화박물관
동국대학교 서울캠퍼스
코엑스
생명의나무 전망대
렛츠런팜 원당
메디플렉스 세종병원
구서울대학교본관
유석 조병옥선생 동상
부천 아트벙커 B39
윤봉길 의사 기념관
고양향교
김포 종합운동장
한양대학교 서울캠퍼스
강동구청
광희문
서초구청
국립 과천 과학관
송도 센트럴 파크 호텔
부천

인창도서관
중랑구청
곤지암 리조트
경희대학교 서울캠퍼스
롯데백화점 일산점
꿈빛 도서관
부천 한국만화영상 진흥원
롯데백화점[[]청량리[]]
베스트웨스턴 하버파크 호텔 인천
현대백화점[[]신촌[]]
늘푸른교회
병암서원
캠프 라과디아 체육공원
JW 메리어트 호텔 서울
광주학생독립운동기념공원(광주학생독립운동역사관, 광주학생독립운동기념탑 포함)
성지파크 골프장
청담대교
문화공원
동부성서침례교회
짜장면박물관
스위스 그랜드 호텔
펄벅 기념관
롯데백화점[[]김포공항[]]
잠실종합운동장
휴띠끄 차이나타운점
양천구청
오라카이 송도 파크 호텔
올림픽공원
웃는얼굴아트센터
신트리공원
남양리 신빈 김씨 묘역
보광사
부천 중동시장
장경사
그랜드백화점 일산점
잠두봉유적
마포 농수산물 시장
천주교장산성당
잇다 스페이스
몽촌토성
갤러리수정
경복궁
잠심대교
용화사
아이파크몰


In [91]:
def replaceRight(original, old, new, count_right):
    repeat=0
    text = original
    old_len = len(old)
    
    count_find = original.count(old)
    if count_right > count_find : # 바꿀 횟수가 문자열에 포함된 old보다 많다면
        repeat = count_find # 문자열에 포함된 old의 모든 개수(count_find)만큼 교체한다
    else :
        repeat = count_right # 아니라면 입력받은 개수(count)만큼 교체한다

    while(repeat):
      find_index = text.rfind(old) # 오른쪽부터 index를 찾기위해 rfind 사용
      text = text[:find_index] + new + text[find_index+old_len:]

      repeat -= 1
      
    return text